# Integración numérica de la ecuación de onda
Utilizaré un método (o más de ser necesario, pues se hinteó que no iba a ser directo) para integrar numéricamente la ecuación de onda adimensionalizada:
$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x)
$$

Para correr este notebook se necesita instalar el paquete GLMakie.jl, que se puede instalar con el comando `]add GLMakie` en la consola de Julia.

## Algoritmo naive:

La idea es usar la clásica discretización de la segunda derivada a ambos lados:

$$
\partial_{zz} f(z) \approx \frac{f(z + h) - 2 f(z) + f(z - h)}{h^2} 
$$

Y con esto, despejar el valor de $u(t + \Delta t, x)$ en función de valores de tiempos anteriores y otras posiciones. Para el primer paso de tiempo simplemente usaré la condición inicial, es decir, supongo velocidad inicial 0 para cada $u(\cdot, x)$. También usaré condiciones de borde fijas.


In [ ]:
using LinearAlgebra
using GLMakie
using Printf
using Integrals

const x_largo = 1.0
const N_x = 201
const x = range(0, x_largo, length=N_x)
const Δx = x[2] - x[1]
const Δt = Δx
const s = (Δt/Δx)^2
const sMatrix = Tridiagonal(ones(N_x-3) .* s, ones(N_x-2) .* 2 .* (1 - s), ones(N_x-3) .* s)
const bordes = 0.0

println("s = ", s)

In [ ]:
function paso_temporal!(u, u_anterior)
    buffer = copy(u)
    u[2:end-1] .= sMatrix * u[2:end-1] .- u_anterior[2:end-1]
    u[1] = bordes
    u[end] = bordes # En verdad esto no es necesario.
    u[2] += s * bordes 
    u[end-1] += s * bordes
    u_anterior .= buffer
end

function actualizar_animacion!(i)
    lines[1][] = Point2f[[x[i], u[i]] for i in 1:N_x] # Horrible sintaxis.
    text.text[] = @sprintf("Energía = %.3f", energía(u, u_anterior))
    paso_temporal!(u, u_anterior)
    return
end

function energía(u, u_anterior)
    dudt = (u[2:end-1] - u_anterior[2:end-1])/Δt 
    dudx = (u[3:end] - u[1:end-2])/(2 * Δx) # Aproximaciones esooo 
    pot_energy_problem = SampledIntegralProblem(dudt.^2, x[2:end-1])  # En esta expresión uso densidad de masa unitaria.
    kin_energy_problem = SampledIntegralProblem(dudx.^2, x[2:end-1])
    method = TrapezoidalRule()
    pot = solve(pot_energy_problem, method)
    kin = solve(kin_energy_problem, method)
    return (pot.u + kin.u)/2.0
end


In [ ]:
# Animación:

u = sin.(2π .* x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 1, -1, 1))
text = text!(ax, 0.03, 0.9, text=@sprintf("Energía = %.3f", energía(u, u_anterior)))
lines = lines!(ax, x, u)
nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "modo_normal.mp4", 1:nframes; framerate=framerate)


Ahora otra condición inicial

In [ ]:
function u_func(x)
    if x < 1/3 || x > 2/3
        return 0.0
    elseif x < 1/2
        return x - 1/3
    else
        return 2/3 - x
    end
end

u = u_func.(x)
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 1, -0.2, 0.2))
lines = lines!(ax, x, u)
text = text!(ax, 0.03, 0.16, text=@sprintf("Energía = %.3f", energía(u, u_anterior)))
nframes = 1080
framerate = 60

record(actualizar_animacion!, fig, "triángulo.mp4", 1:nframes; framerate=framerate)


Adjuntaré un análisis de estabilidad que hice, que me asegura que cuando $ s\leq 1$ el método es estable. También vemos que para $s<1$, la condición inicial no se rearma en un periodo. Esto sólo puede ocurrir si la relación de dispersión numérica es distinta a la exacta (que es $\omega = k$). Vemos que hay unas oscilaciones extrañas ahí metidas cuando $s = 1$, una especie de ruido (proveniente del modo más pequeño posible debido a la discretización) que me dificulta calcular la energía, dandome valores muy oscilantes. Sin embargo, está claro que la energía efectivamente se conserva pues al cabo de varias oscilaciones, la condición inicial se rearma.

## Corrección no lineal

Agregamos ahora la primera corrección no-lineal de la ecuación. La ecuación de onda no lineal es:

$$
\partial_{tt} u(t, x) = \partial_{xx} u(t, x) (1 + 2 \partial_x u(t, x)^2)
$$

Esto se puede discretizar de la misma forma que antes, pero ahora hay que calcular la derivada espacial de $u$ en cada paso de tiempo. Para esto, usaré un esquema de diferencias finitas centradas.

$$
\partial_x u(t, x) \approx \frac{u(t, x + h) - u(t, x - h)}{2h}
$$


In [ ]:
function paso_temporal_no_lineal!(u, u_anterior)
    buffer = copy(u)
    u[2:end-1] .= sMatrix * u[2:end-1] .- u_anterior[2:end-1]
    u[2:end-1] .-= 1 .* ((u[3:end] .- u[1:end-2])./(2*Δx)).^2 .* s .* (u[3:end] .- 2 .* u[2:end-1] .+ u[1:end-2])
    u[1] = bordes
    u[end] = bordes # En verdad esto no es necesario.
    u[2] += s * bordes 
    u[end-1] += s * bordes
    u_anterior .= buffer
end

function actualizar_animacion_no_lineal!(i)
    lines[1][] = Point2f[[x[i], u[i]] for i in 1:N_x] # Horrible sintaxis.
    paso_temporal_no_lineal!(u, u_anterior)
    return
end

In [ ]:
u = sin.(π .* x) ./ 50 # Debo dividir para estar en la región de validez.
u_anterior = copy(u)
fig = Figure()
ax = Axis(fig[1, 1], limits=(0, 1, -1/50., 1/50))
lines = lines!(ax, x, u)
nframes = 60*10
framerate = 60

record(actualizar_animacion_no_lineal!, fig, "modo_normal_no_lineal.mp4", 1:nframes; framerate=framerate)

### Comentarios:

Para la corrección no lineal, noto que mi ecuación explota para condidiones inicales con derivadas muy grandes (lo cual es de esperar, pues no estoy incluyendo todos los términos del taylor y ya no puedo confiar en la linealidad para hacer sentido de condiciones con derivadas grandes).

La verdad quizás sea otro tipo de corrección no lineal la que se buscaba, como la del experimento de Fermi-Pasta-Ulam-Tsingou. La corrección no lineal de este caso vendría de términos no lineales en los resortes que unen las masas del modelo discreto, lo cual no es lo que estoy haciendo aquí. De hecho, mantuve restringido el movimiento en x para derivar esta ecuación, lo cual igual es fuerte y me restringe a regiones de validez pequeñas.